In [1]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('lab09.ok')
try:
    _ = ok.auth(inline=True)
except:
    _ = ok.auth(inline=True, force=True)

Assignment: lab09
OK, version v1.14.15

Successfully logged in as togden1@binghamton.edu


## Lab 9: Pandas Continued

**This lab is due Thursday, June 27 at 11:59 PM.**

We will continue discussion of [Pandas](https://pandas.pydata.org/). In this lab, you will learn about:

* Grouping dataframes
* Merging dataframes

You will also be working with a dataset from the City of Berkeley containing data on calls to the Berkeley Police Department. Information about the dataset can be found [at this link](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5).

**This assignment should be completed and submitted before Tuesday, November 5, 2019 at 01:59 AM.**

In [2]:
# Setup from Last Lab

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import requests
import zipfile
from pathlib import Path
%matplotlib inline

import seaborn as sns

def fetch_and_cache(data_url, file, data_dir="data", force=False):
    """
    Download and cache a url and return the file object.
    
    data_url: the web address to download
    file: the file in which to save the results.
    data_dir: (default="data") the location to save the data
    force: if true the file is always re-downloaded 
    
    return: The pathlib.Path to the file.
    """
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok=True)
    file_path = data_dir/Path(file)
    if force and file_path.exists():
        file_path.unlink()
    if force or not file_path.exists():
        print('Downloading...', end=' ')
        resp = requests.get(data_url)
        with file_path.open('wb') as f:
            f.write(resp.content)
        print('Done!')
    else:
        import time 
        created = time.ctime(file_path.stat().st_ctime)
        print("Using cached version downloaded at", created)
    return file_path

data_url = 'https://www.ssa.gov/oact/babynames/state/namesbystate.zip'
namesbystate_path = fetch_and_cache(data_url, 'namesbystate.zip')
zf = zipfile.ZipFile(namesbystate_path, 'r')

column_labels = ['State', 'Sex', 'Year', 'Name', 'Count']

def load_dataframe_from_zip(zf, f):
    with zf.open(f) as fh: 
        return pd.read_csv(fh, header=None, names=column_labels)

states = [
    load_dataframe_from_zip(zf, f)
    for f in sorted(zf.filelist, key=lambda x:x.filename) 
    if f.filename.endswith('.TXT')
]

baby_names = states[0]
for state_df in states[1:]:
    baby_names = pd.concat([baby_names, state_df])
baby_names = baby_names.reset_index().iloc[:, 1:]
ca = baby_names[baby_names['State'] == 'CA']

Downloading... Done!


Note that after activating matplotlib to display figures inline via the IPython magic `%matplotlib inline`, we configure a custom default figure size. Virtually every default aspect of matplotlib [can be customized](https://matplotlib.org/users/customizing.html).

In [6]:
plt.rcParams['figure.figsize'] = (12, 9)
ca.head()

State Sex  Year      Name  Count
379228    CA   F  1910      Mary    295
379229    CA   F  1910     Helen    239
379230    CA   F  1910   Dorothy    220
379231    CA   F  1910  Margaret    163
379232    CA   F  1910   Frances    134


## Part 1: Data Aggregration (Grouping Data Frames)

### Question 1a
To count the number of instances of each unique value in a `Series`, we can use the `value_counts()` [method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) as `df["col_name"].value_counts()`. Count the number of different names for each Year in `CA` (California).  (You may use the `ca` DataFrame created above.)

**Note:** *We are not computing the number of babies but instead the number of names (rows in the table) for each year.*

**Note:** *Because the data set may have been updated since the tests were written, it is possible that the counts from the tests are slightly different from your calculations. Ignore those tests savely. We will grade based on the correct counts.*

<!--
BEGIN QUESTION
name: q1a
-->

In [7]:
num_of_names_per_year = ca['Name'].value_counts()
num_of_names_per_year[:]

Jean         215
Francis      213
Guadalupe    211
Marion       210
Jessie       209
Leslie       202
Lee          202
Frankie      193
Jackie       192
Cruz         191
Trinidad     191
Noel         191
Robin        190
Terry        183
Robert       182
John         181
Michael      179
Adrian       179
Billie       177
Lynn         177
Angel        176
James        176
Lupe         176
Jesus        175
Richard      175
David        174
Rene         172
Sidney       171
Jerry        171
Mary         170
            ... 
Abanoub        1
Sadhana        1
Lelan          1
Lafondra       1
Mairead        1
Delorean       1
Josejr         1
Jilliane       1
Niyla          1
Xaviera        1
Denishia       1
Zanna          1
Kailo          1
Guadelupe      1
Krisna         1
York           1
Syanna         1
Rishit         1
Lanyah         1
Miangel        1
Dafina         1
Eulogio        1
Shivali        1
Aanchal        1
Tallen         1
Kelechi        1
Sieanna        1
Radiah        

In [8]:
ok.grade("q1a");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
q1a > Suite 1 > Case 1

>>> num_of_names_per_year[2007]
55

# Error: expected
#     7250
# but got
#     55

Run only this test case with "python3 ok -q q1a --suite 1 --case 1"
---------------------------------------------------------------------
Test summary
    Passed: 0
    Failed: 1
[k..........] 0.0% passed



### Question 1b
Using the same approach as above, count the number of different names for each gender in `CA`.


<!--
BEGIN QUESTION
name: q1b
-->

In [90]:
num_of_names_per_gender = ca.groupby(['Sex','Name']).count().drop(['State','Year'],axis=1)

In [91]:
ok.grade("q1b");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
q1b > Suite 1 > Case 1

>>> num_of_names_per_gender["F"]
Traceback (most recent call last):
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\frame.py", line 2688, in __getitem__
    return self._getitem_column(key)
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\frame.py", line 2695, in _getitem_column
    return self._get_item_cache(key)
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\generic.py", line 2489, in _get_item_cache
    values = self._data.get(item)
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\internals.py", line 4115, in get
    loc = self.items.get_loc(item)
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(self._maybe_cast_indexer(key))
  File "pandas\_libs\index.pyx", line 140, in pa

### Question 2: Groupby ###

Before we jump into using the [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) function in Pandas, let's recap how grouping works in general for tabular data through a guided set of questions based on a small toy dataset of movies and genres. 

**Note:** If you want to see a visual of how grouping of data works, here is a link to an animation [Groupby Animation](http://www.ds100.org/sp18/assets/lectures/lec03/03-groupby_and_pivot.pdf)

**Problem Setting:** This summer 2018, there were a lot of good and bad movies that came out. Below is a dataframe with 5 columns: name of the movie as a `string`, the genre of the movie as a `string`, the first name of the director of the movie as a `string`, the average rating out of 10 on Rotten Tomatoes as an `integer`, and the total gross revenue made by the movie as an `integer`. The point of these guided questions (parts a and b) below is to understand how grouping of data works in general, **not** how grouping works in code. We will worry about how grouping works in Pandas in 7c, which will follow.

Below is the `movies` dataframe we are using, imported from the `movies.csv` file located in the `lab02` directory.

In [21]:
movies = pd.read_csv("movies.csv")
movies

director                      genre  \
0         David         Action & Adventure   
1          Bill                     Comedy   
2           Ron  Science Fiction & Fantasy   
3      Baltasar                      Drama   
4          Bart                      Drama   
5          Gary         Action & Adventure   
6          Drew         Action & Adventure   
7          Brad                  Animation   
8          Jeff                     Comedy   
9          J.A.  Science Fiction & Fantasy   
10      Charles                     Comedy   
11       Gerard                     Horror   
12       Peyton         Action & Adventure   
13       Genndy                  Animation   
14       Rawson         Action & Adventure   
15           Ol                     Comedy   
16  Christopher         Action & Adventure   
17         Marc                     Comedy   

                                    movie  rating    revenue  
0                              Deadpool 2       7  318344544  
1                               Book Club       5   68566296  
2                 Solo: A Star Wars Story       6  213476293  
3                                  Adrift       6   31445012  
4                        American Animals       6    2847319  
5                                Oceans 8       6  138803463  
6                           Hotel Artemis       8    6708147  
7                           Incredibles 2       5  594398019  
8                                     Tag       6   54336863  
9          Jurassic World: Fallen Kingdom       6  411873505  
10                             Uncle Drew       5   42201656  
11                        The First Purge       7   68765655  
12                   Ant-Man and the Wasp       5  208681866  
13  Hotel Transylvania 3: Summer Vacation       5  154418311  
14                             Skyscraper       6   66801215  
15            Mamma Mia! Here We Go Again       8  111705055  
16            Mission: Impossible-Fallout       6  182080372  
17                     Christopher Robbin       6    6786317

### Question 2a

If we grouped the `movies` dataframe above by `genre`, how many groups would be in the output and what would be the groups? Assign `num_groups` to the number of groups created (hard-code) and fill in `genre_list` as a list containing the names of unique genres as strings that represent the groups.

<!--
BEGIN QUESTION
name: q2a
-->

In [31]:
num_groups = 6
genre_list = movies['genre'].unique()
genre_list

array(['Action & Adventure', 'Comedy', 'Science Fiction & Fantasy',
       'Drama', 'Animation', 'Horror'], dtype=object)

In [32]:
ok.grade("q2a");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 2
    Failed: 0
[ooooooooook] 100.0% passed



### Question 2b

Whenever we group tabular data, it is usually the case that we need to aggregate values from the ungrouped column(s). If we were to group the `movies` dataframe above by `genre`, which column(s) in the `movies` dataframe would it make sense to aggregate if we were interested in finding how well each genre did in the eyes of people? Fill in `agg_cols` with a list of column name or names.

<!--
BEGIN QUESTION
name: q2b
-->

In [33]:
agg_cols = ['rating', 'revenue']

In [34]:
ok.grade("q2b");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



Now, let's see `groupby` in action, instead of keeping everything abstract. To aggregate data in Pandas, we use the `.groupby()` [function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html). The code below will group the `movies` dataframe by `genre` and find the average revenue and rating for each genre. You can verify you had the same number of groups as what you answered in 7a. 

In [45]:
movies.loc[:, ['genre', 'rating', 'revenue']].groupby('genre').mean()

rating      revenue
genre                                           
Action & Adventure         6.333333  153569934.5
Animation                  5.000000  374408165.0
Comedy                     6.000000   56719237.4
Drama                      6.000000   17146165.5
Horror                     7.000000   68765655.0
Science Fiction & Fantasy  6.000000  312674899.0

### Question 2c

Let's move back to baby names and specifically, the `ca` dataframe. Find the sum of `Count` for each `Name` in the `ca` table. You should use `df.groupby("col_name").sum()`. Your result should be a Pandas Series.

**Note:** *In this question we are now computing the number of registered babies with a given name.*

**Note:** *The numbers in the test are slightly off. Don't worry about them if the discrepancy is not too large.*


<!--
BEGIN QUESTION
name: q2c
-->

In [66]:
count_for_names = ca.groupby('Name').sum()
count_for_names.sort_values('Count',ascending=False)[:5]
count_for_names.head()

Year  Count
Name                 
Aadan     6031     18
Aaden    24150    627
Aadhav    6046     21
Aadhira   4035     16
Aadhya   20132    292

In [59]:
ok.grade("q2c");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
q2c > Suite 1 > Case 1

>>> count_for_names["Michael"]
Traceback (most recent call last):
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\frame.py", line 2688, in __getitem__
    return self._getitem_column(key)
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\frame.py", line 2695, in _getitem_column
    return self._get_item_cache(key)
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\generic.py", line 2489, in _get_item_cache
    values = self._data.get(item)
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\internals.py", line 4115, in get
    loc = self.items.get_loc(item)
  File "C:\Users\Tim\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(self._maybe_cast_indexer(key))
  File "pandas\_libs\index.pyx", line 140, in pand

### Question 2d

Find the sum of `Count` for each female name after year 1999 (`>1999`) in California.


<!--
BEGIN QUESTION
name: q2d
-->

In [68]:
female_name_count = ca.loc[].groupby('Name').sum()
female_name_count.sort_values(ascending=False)[:5]

KeyError: 'the label [True] is not in the [index]'

In [ ]:
ok.grade("q2d");

### Question 3: Grouping Multiple Columns

Let's move back to the `movies` dataframe. Which of the following lines of code will output the following dataframe? Write your answer (hard-coded) as either 1, 2, 3, or 4. Recall that the arguments to `pd.pivot_table` are as follows: `data` is the input dataframe, `index` includes the values we use as rows, `columns` are the columns of the pivot table, `values` are the values in the pivot table, and `aggfunc` is the aggregation function that we use to aggregate `values`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>rating</th>
      <th>5</th>
      <th>6</th>
      <th>7</th>
      <th>8</th>
    </tr>
    <tr>
      <th>genre</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Action &amp; Adventure</th>
      <td>208681866.0</td>
      <td>129228350.0</td>
      <td>318344544.0</td>
      <td>6708147.0</td>
    </tr>
    <tr>
      <th>Animation</th>
      <td>374408165.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Comedy</th>
      <td>55383976.0</td>
      <td>30561590.0</td>
      <td>NaN</td>
      <td>111705055.0</td>
    </tr>
    <tr>
      <th>Drama</th>
      <td>NaN</td>
      <td>17146165.5</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Horror</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>68765655.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Science Fiction &amp; Fantasy</th>
      <td>NaN</td>
      <td>312674899.0</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>

1) `pd.pivot_table(data=movies, index='genre', columns='rating', values='revenue', aggfunc=np.mean)`

2) `movies.groupby(['genre', 'rating'])['revenue'].mean()`

3) `pd.pivot_table(data=movies, index='rating', columns='genre', values='revenue', aggfunc=np.mean)`

4) `movies.groupby('revenue')[['genre', 'rating']].mean()`


<!--
BEGIN QUESTION
name: q3
-->

In [71]:
q3_answer = 1

In [72]:
ok.grade("q3");

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



### Question 4: Merging


#### Question 4a

Time to put everything together! Merge `movies` and `count_for_names` to find the number of registered baby names for each director using [`pd.merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html?highlight=merge#pandas.merge). Only include names that appear in both `movies` and `count_for_names`.

**Hint:** You might need to convert the `count_for_names` series to a dataframe. Take a look at the ``to_frame`` method of a series to do this. 

Your first row should look something like this:

**Note**: It is ok if you have 2 separate columns with names instead of just one column.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>director</th>
      <th>genre</th>
      <th>movie</th>
      <th>rating</th>
      <th>revenue</th>
      <th>Count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>David</td>
      <td>Action &amp; Adventure</td>
      <td>Deadpool 2</td>
      <td>7</td>
      <td>318344544</td>
      <td>371646</td>
    </tr>
  </tbody>
</table>
</table>


<!--
BEGIN QUESTION
name: q4a
-->

In [92]:
merged_df = pd.merge(movies, count_for_names, on='movie')
merged_df.head()

KeyError: 'movie'

In [ ]:
ok.grade("q4a");

#### Question 4b

How many directors in the original `movies` table did not get included in the `merged_df` dataframe? Please hard-code your answer as a number in `q4b`, then explain your answer in 1-2 sentences as a comment below.


<!--
BEGIN QUESTION
name: q4b
-->

In [ ]:
q_4b = ...

# Explain your solution: ...

In [ ]:
ok.grade("q4b");

# Part 2: Cleaning and Exploring the Data

To retrieve the dataset, we will use the `ds100_utils.fetch_and_cache` utility.

In [93]:
import ds100_utils

data_dir = 'data'
data_url = 'http://ids.binghamton.edu/datasets/lab09_data_fa19.zip'
file_name = 'lab09_data_fa19.zip'

dest_path = ds100_utils.fetch_and_cache(data_url=data_url, file=file_name, data_dir=data_dir)
print(f'Located at {dest_path}')

Using cached version that was downloaded (UTC): Tue Oct 29 11:09:47 2019
Located at data\lab09_data_fa19.zip


We will now directly unzip the ZIP archive and start working with the uncompressed files.

Note: There is no single right answer regarding whether to work with compressed files in their compressed state or to uncompress them on disk permanently. If you for example need to work with multiple tools on the same files, or write many notebooks to analyze them, and they are not too large, it may be more convenient to uncompress them once.  But you may also have situations where you find it preferable to work with the compressed data directly.  

Python gives you tools for both approaches, and you should know how to perform both tasks in order to choose the one that best suits the problem at hand.

---

Run the cell below to extract the zip file into the data directory.

In [94]:
import zipfile

my_zip = zipfile.ZipFile(dest_path, 'r')
my_zip.extractall(data_dir)

Now, we'll use a method of the `Pathlib.Path` class called `glob` to list all files in the `data` directory. You will find useful information in pathlib [docs](https://docs.python.org/3/library/pathlib.html).

Below, we use pathlib's `glob` method to store the list of all files' names from the `data_dir` directory in the variable `file_names`. These names should be strings that contain only the file name (e.g. `dummy.txt` not `data/dummy.txt`). The asterisk (*) character is used with the `glob` method to match any string.

In [95]:
from pathlib import Path
data_dir_path = Path('data') # creates a Path object that points to the data directory
file_names = [x.name for x in data_dir_path.glob('*') if x.is_file()]
file_names

['ben_kurtovic.py',
 'Berkeley_PD_-_Calls_for_Service.csv',
 'do_not_readme.md',
 'dummy.txt',
 'hello_world.py',
 'lab09_data_fa19.zip',
 'namesbystate.zip']

Let's now load the CSV file we have into a `pandas.DataFrame` object.

In [ ]:
calls = pd.read_csv("data/Berkeley_PD_-_Calls_for_Service.csv")
calls.head()

We see that the fields include a case number, the offense type, the date and time of the offense, the "CVLEGEND" which appears to be related to the offense type, a "CVDOW" which has no apparent meaning, a date added to the database, and the location spread across four fields.

Let's also check some basic information about these files using the `DataFrame.describe` and `DataFrame.info` methods.

In [ ]:
calls.info()
calls.describe()

Notice that the functions above reveal type information for the columns, as well as some basic statistics about the numerical columns found in the DataFrame. However, we still need more information about what each column represents. Let's explore the data further in Question 5.

Before we go over the fields to see their meanings, the cell below will verify that all the events happened in Berkeley by grouping on the `City` and `State` columns. You should see that all of our data falls into one group.

In [ ]:
calls.groupby(["City","State"]).count()

## Question 5
Above, when we called `head`, it seemed like `OFFENSE` and `CVLEGEND` both contained information about the type of event reported. What is the difference in meaning between the two columns? One way to probe this is to look at the `value_counts` for each Series.

In [ ]:
calls['OFFENSE'].value_counts().head(10)

In [ ]:
calls['CVLEGEND'].value_counts().head(10)

### Question 5a

Above, it seems like `OFFENSE` is more specific than `CVLEGEND`, e.g. "LARCENY" vs. "THEFT FELONY (OVER $950)". For those of you who don't know the word "larceny", it's a legal term for theft of personal property.

To get a sense of how many subcategories there are for each `OFFENSE`, set `calls_by_cvlegend_and_offense` equal to a multi-indexed series where the data is first indexed on the `CVLEGEND` and then on the `OFFENSE`, and the data is equal to the number of offenses in the database that match the respective `CVLEGEND` and `OFFENSE`. For example, calls_by_cvlegend_and_offense["LARCENY", "THEFT FROM PERSON"] should return 24.

<!--
BEGIN QUESTION
name: q5a
-->

In [ ]:
calls_by_cvlegend_and_offense = ...
calls_by_cvlegend_and_offense

In [ ]:
ok.grade("q5a");

### Question 5b

In the cell below, set `answer5b` equal to a list of strings corresponding to the possible values for `OFFENSE` when `CVLEGEND` is "LARCENY". You can type the answer manually, or you can create an expression that automatically extracts the names.

<!--
BEGIN QUESTION
name: q1b
-->

In [ ]:
# You may use this cell for your scratch work as long as you enter
# in your final answers in the answer1 variable.
answer5b = ...

In [ ]:
ok.grade("q5b");

## Question 6

What are the five crime types of CVLEGEND that have the most crime events? You may need to use `value_counts` to find the answer.
Save your results into `answer6` as a list of strings.

**Hint:** *The `keys` method of the Series class might be useful.*

<!--
BEGIN QUESTION
name: q2
-->

In [ ]:
answer6 = ...
answer6

In [ ]:
ok.grade("q6");

# Part 3: Visualizing the Data

## Pandas vs. Seaborn Plotting

Pandas offers basic functionality for plotting. For example, the `DataFrame` and `Series` classes both have a `plot` method. However, the basic plots generated by pandas are not particularly pretty. While it's possible to manually use matplotlib commands to make pandas plots look better, we'll instead use a high level plotting library called Seaborn that will take care of most of this for us.

As you learn to do data visualization, you may find the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) and [Seaborn documentation](https://seaborn.pydata.org/api.html) helpful!

As an example of the built-in plotting functionality of pandas, the following example uses `plot` method of the `Series` class to generate a `barh` plot type to visually display the value counts for `CVLEGEND`.

In [ ]:
ax = calls['CVLEGEND'].value_counts().plot(kind='barh')
ax.set_ylabel("Crime Category")
ax.set_xlabel("Number of Calls")
ax.set_title("Number of Calls By Crime Type");

By contrast, the Seaborn library provides a specific function `countplot` built for plotting counts. It operates directly on the DataFrame itself i.e. there's no need to call `value_counts()` at all. This higher level approach makes it easier to work with. Run the cell below, and you'll see that the plot is much prettier (albeit in a weird order).

In [ ]:
ax = sns.countplot(data=calls, y="CVLEGEND")
ax.set_ylabel("Crime Category")
ax.set_xlabel("Number of Calls")
ax.set_title("Number of Calls By Crime Type");

If we want the same ordering that we had in the pandas plot, we can use the order parameter of the `countplot` method. It takes a list of strings corresponding to the axis to be ordered. By passing the index of the `value_counts`, we get the order we want.

In [ ]:
ax = sns.countplot(data=calls, y="CVLEGEND", order=calls["CVLEGEND"].value_counts(ascending=True).index);
ax.set_ylabel("Crime Category")
ax.set_xlabel("Number of Calls")
ax.set_title("Number of Calls By Crime Type");

Voilà! Now we have a pretty bar plot with the bars ordered by size. Though `seaborn` appears to provide a superior plot from an aesthetic point of view, the `pandas` plotting library is also good to understand. You'll get practice using both libraries in the following questions.

## An Additional Note on Plotting in Jupyter Notebooks

You may have noticed that many of our code cells involving plotting end with a semicolon (;). This prevents any extra output from the last line of the cell that we may not want to see. Try adding this to your own code in the following questions!

## Question 7

Now it is your turn to make some plots using `pandas` and `seaborn`. Let's start by looking at the distribution of calls over days of the week.

The CVDOW field isn't named helpfully and it is hard to see the meaning from the data alone. According to the website linked at the top of this notebook, CVDOW is actually indicating the day that events happened. 0->Sunday, 1->Monday ... 6->Saturday. 

### Question 7a

Add a new column `Day` into the `calls` dataframe that has the string weekday (eg. 'Sunday') for the corresponding value in CVDOW. For example, if the first 3 values of `CVDOW` are `[3, 6, 0]`, then the first 3 values of the `Day` column should be `["Wednesday", "Saturday", "Sunday"]`.

**Hint:** *Try using the [Series.map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function on `calls["CVDOW"]`.  Can you assign this to the new column `calls["Day"]`?*

```
BEGIN QUESTION
name: q7a
```

In [ ]:
days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
day_indices = range(7)
indices_to_days_dict = dict(zip(day_indices, days)) # Should look like {0:"Sunday", 1:"Monday", ..., 6:"Saturday"}
...

In [ ]:
ok.grade("q7a");

### Question 7b

Run the cell below to create a `seaborn` plot. This plot shows the number of calls for each day of the week. Notice the use of the `rotation` argument in `ax.set_xticklabels`, which rotates the labels by 90 degrees.

In [ ]:
ax = sns.countplot(data=calls, x='Day', order=days)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_ylabel("Number of Calls")
ax.set_title("Number of Calls For Each Day of the Week");

Now, let's make the same plot using `pandas`. Construct a vertical bar plot with the count of the number of calls (entries in the table) for each day of the week **ordered by the day of the week** (eg. `Sunday`, `Monday`, ...). Do not use `sns` for this plot. Be sure that your axes are labeled and that your plot is titled.

**Hint:** *Given a series `s`, and an array `coolIndex` that has the same entries as in `s.index`, `s[coolIndex]` will return a copy of the series in the same order as `coolIndex`.*

<!--
BEGIN QUESTION
name: q3b
-->

In [ ]:
...

# Leave this for grading purposes
ax_7b = plt.gca()

In [ ]:
ok.grade("q7b");

## Question 8

It seems weekdays generally have slightly more calls than Saturday or Sunday, but the difference does not look significant.  

We can break down into some particular types of events to see their distribution. For example, let's make a bar plot for the CVLEGEND "NOISE VIOLATION". Which day is the peak for "NOISE VIOLATION"?

### Question 8a

This time, use `seaborn` to create a vertical bar plot of the number of total noise violations reported on each day of the week, again ordered by the days of the week starting with Sunday. Do not use `pandas` to plot.

**Hint:** *If you're stuck, use the code for the seaborn plot in Question 7b as a starting point.*

<!--
BEGIN QUESTION
name: q8a
-->

In [ ]:
...

# Leave this for grading purposes
ax_8a = plt.gca()

In [ ]:
ok.grade("q8a");

### Question 8b

Do you realize anything interesting about the distribution of NOISE VIOLATION calls over a week? Type a 1-sentence answer in the cell below.

## Question 9

Let's look at a similar distribution but for a crime we have much more calls data about. In the cell below, create the same plot as you did in Question 8, but now looking at instances of the CVLEGEND "FRAUD" (instead of "NOISE VIOLATION"). Use either `pandas` or `seaborn` plotting as you desire.

<!--
BEGIN QUESTION
name: q9
-->

In [ ]:
...

# Leave this for grading purposes
ax_9 = plt.gca()

In [ ]:
ok.grade("q9");

## Question 10

### Question 10a

Now let's look at the EVENTTM column which indicates the time for events. Since it contains hour and minute information, let's extract the hour info and create a new column named `Hour` in the `calls` dataframe. You should save the hour as an `int`. Then plot the frequency of each hour in the table (i.e., `value_counts()`) sorted by the hour of the day (i.e., `sort_index()`).

You will want to look into how to use:

* [Series.str.slice](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.slice.html#pandas.Series.str.slice) to select the substring.
* [Series.astype](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html) to change the type.

**Hint:** *The `str` helper member of a series can be used to grab substrings.  For example, `calls["EVENTTM"].str.slice(3,5)` returns the minute of each hour of the `EVENTTM`.*

<!--
BEGIN QUESTION
name: q10
-->

In [ ]:
...

In [ ]:
ok.grade("q10");

The code in the cell below creates a pandas bar plot showing the number of FRAUD crimes committed at each hour of the day.

In [ ]:
ax = calls[calls["CVLEGEND"] == "FRAUD"]['Hour'].value_counts().sort_index().plot(kind='bar')
ax.set_xlabel("Hour of the Day")
ax.set_ylabel("Number of Calls")
ax.set_title("Number of Calls Reporting Fraud For Each Day of the Week");

The cell below contains a seaborn plot of the same data.

In [ ]:
ax = sns.countplot(calls[calls["CVLEGEND"] == "FRAUD"]['Hour'])
ax.set_xlabel("Hour of the Day")
ax.set_ylabel("Number of Calls")
ax.set_title("Number of Calls Reporting Fraud For Each Day of the Week");
#alternate solution: sns.countplot(data=calls[calls["CVLEGEND"] == "FRAUD"], x = 'Hour');

### Question 10b

According to our plots, there seems to be a spike in calls reporting fraud at midnight. Do you trust that this spike is legitimate, or could there be an issue with our data? Explain your reasoning in 1-2 sentences in the cell below.

## Question 11 (OPTIONAL) Will not be graded.

In the cell below, we generate a boxplot which examines the hour of day of each crime broken down by the `CVLEGEND` value.  To construct this plot we used the [DataFrame.boxplot](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.boxplot.html) documentation.

In [ ]:
calls.boxplot(column="Hour", by='CVLEGEND', rot=90);

While the pandas boxplot is informative, we can use seaborn to create a more visually-appealing plot. Using seaborn, regenerate a better box plot. See either the [textbook PTDS](https://www.textbook.ds100.org/ch/06/viz_quantitative.html) or the [seaborn boxplot documentation](https://seaborn.pydata.org/generated/seaborn.boxplot.html).

Looking at your plot, which crime type appears to have the largest interquartile range? Put your results into `answer11` as a string.


In [ ]:
answer11 = ... #your answer here

# Todo: Make a boxplot with seaborn
...

## Congratulations

Congrats! You are finished with this assignment.

In [ ]:
# Save your notebook first, then run this cell to submit.
ok.submit()

Go to okpy.org and check if your submission was successful. You have the option to manually upload the file (the .ipynb file).